In [ ]:
import dotenv
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import time
from typing import List, Dict

dotenv.load_dotenv()  # .env 값 읽어오기

key = os.getenv('FARMER_KEY')
base_url = f"http://211.237.50.150:7080/openapi/{key}/xml/Grid_20171122000000000552_1"
print(base_url[:40])import dotenv
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import time
from typing import List, Dict

dotenv.load_dotenv()  # .env 값 읽어오기

key = os.getenv('FARMER_KEY')
base_url = f"http://211.237.50.150:7080/openapi/{key}/xml/Grid_20171122000000000552_1"
print(base_url[:40])

In [ ]:
import requests
start_index=1; end_index=10
base_url = f"http://211.237.50.150:7080/openapi/{key}\
/xml/Grid_20171122000000000552_1/{start_index}/{end_index}"
response = requests.get(base_url, timeout=10)
response.raise_for_status()
# XML 파싱
root = ET.fromstring(response.content)
# 출력결과를 확인
# xml_string = ET.tostring(root, encoding='unicode')
# print(xml_string)
# 전체 데이터 개수 확인  
total_count_elem = root.find('totalCnt')
total_count = int(total_count_elem.text) if total_count_elem is not None else 0
print(total_count) 

data_list = []
for row in root.findall('row'):
    item = {}
    for child in row:
        item[child.tag] = child.text
    data_list.append(item)

print(f"  → 수집 완료: {len(data_list)}개 (전체: {total_count}개)")

In [ ]:
# 데이터 추출
import time
batch_size = 10  # 10개씩 
current_index = batch_size+1   #11 ~ 20  21 ~ 30  31 ~ 40  450   451 451
delay = 0.5

while current_index <= total_count:
    time.sleep(delay)
    end_index = min(current_index+batch_size-1, total_count)

    base_url = f"http://211.237.50.150:7080/openapi/{key}\
    /xml/Grid_20171122000000000552_1/{current_index}/{end_index}"
    response = requests.get(base_url, timeout=10)
    response.raise_for_status()
    # XML 파싱
    root = ET.fromstring(response.content)
    for row in root.findall('row'):
        item = {}
        for child in row:
            item[child.tag] = child.text
        data_list.append(item)
    print(f"  → 수집 완료: {len(data_list)}개 (전체: {total_count}개)")

    current_index = end_index + 1

In [ ]:
def collect_all_data(self, batch_size: int = 10, delay: float = 0.5):
        """
        API에서 전체 데이터를 자동으로 수집합니다.
        
        이 메서드는 클래스의 핵심 기능으로, 페이징을 자동으로 처리합니다.
        
        작동 과정:
        1. 첫 번째 요청 (1~batch_size)으로 전체 데이터 개수 파악
        2. 전체 개수를 기준으로 필요한 요청 횟수 계산
        3. 반복문으로 모든 데이터 수집
        4. 각 요청 사이에 delay 시간만큼 대기 (서버 부하 방지)
        
        Args:
            batch_size (int, optional): 한 번에 가져올 데이터 개수
                기본값: 10
                권장값: 10~100 (너무 크면 API 오류 발생 가능)
                예: batch_size=10이면 1~10, 11~20, 21~30... 순서로 요청
                
            delay (float, optional): 요청 간 대기 시간 (초)
                기본값: 0.5초
                권장값: 0.3~1.0초 (너무 짧으면 서버가 차단할 수 있음)
                
        처리 로직 예시:
            전체 데이터 100개, batch_size=10인 경우:
            1차: 1~10 요청 → 10개 수집
            2차: 11~20 요청 → 10개 수집
            ...
            10차: 91~100 요청 → 10개 수집
            총 100개 수집 완료
        
        사용 예:
            >>> collector.collect_all_data(batch_size=20, delay=1.0)
            ============================================================
            텃밭 정보 API 데이터 수집 시작
            ============================================================
            요청 중: 1 ~ 20
              → 수집 완료: 20개 (전체: 1234개)
            요청 중: 21 ~ 40
              → 수집 완료: 20개 (전체: 1234개)
            ...
        
        재시도 로직:
            - 데이터 수집 실패 시 2초 대기 후 재시도
            - 무한 재시도하므로 수동으로 중단해야 함 (Ctrl+C)
        """
        print("=" * 60)
        print("텃밭 정보 API 데이터 수집 시작")
        print("=" * 60)
        
        # 1단계: 첫 번째 요청으로 전체 데이터 개수 확인
        first_data, total_count = self.fetch_data(1, batch_size)
        
        # 데이터가 없으면 종료
        if total_count == 0:
            print("\n수집할 데이터가 없습니다.")
            return
        
        # 첫 번째 배치 데이터 저장
        self.all_data.extend(first_data)
        
        # 2단계: 나머지 데이터 수집
        # 다음 시작 위치 계산
        current_idx = batch_size + 1
        
        # 전체 데이터를 다 가져올 때까지 반복
        while current_idx <= total_count:
            # API 서버 부하 방지를 위한 대기
            time.sleep(delay)
            
            # 종료 인덱스 계산
            # 예: current_idx=91, batch_size=10, total_count=95
            #     → end_idx = min(100, 95) = 95
            end_idx = min(current_idx + batch_size - 1, total_count)
            
            # 데이터 요청
            data_list, _ = self.fetch_data(current_idx, end_idx)
            
            # 실패 시 재시도
            if not data_list:
                print(f"  → 데이터 수집 실패. 다시 시도...")
                time.sleep(2)
                continue  # 같은 범위를 다시 요청
            
            # 성공 시 데이터 추가 및 다음 범위로 이동
            self.all_data.extend(data_list)
            current_idx = end_idx + 1
        
        
import xml.etree.ElementTree as ET
import pandas as pd
import time
from typing import List, Dict

class FarmAPICollector:
    """
    텃밭 정보 API 데이터 수집 클래스
    
    이 클래스는 공공데이터 API에서 텃밭 정보를 자동으로 수집합니다.
    
    주요 기능:
    - API 호출 및 XML 데이터 파싱
    - 전체 데이터 자동 수집 (페이징 처리)
    - CSV/Excel 파일로 저장
    - 데이터 요약 통계 제공
    
    속성:
        base_url (str): API의 기본 URL
        all_data (list): 수집된 모든 데이터를 저장하는 리스트
    
    사용 예:
        >>> collector = FarmAPICollector("http://api주소")
        >>> collector.collect_all_data()  # 전체 데이터 수집
        >>> collector.save_to_csv("결과.csv")  # CSV로 저장
    """
    
    def __init__(self, base_url: str):
        """
        FarmAPICollector 초기화
        
        Args:
            base_url (str): API 기본 URL
                예: "http://211.237.50.150:7080/openapi/sample/xml/Grid_20171122000000000552_1"
                주의: URL 끝에 슬래시(/)를 붙이지 마세요!
        
        초기화 후 상태:
            - self.base_url: 제공된 API URL 저장
            - self.all_data: 빈 리스트로 초기화 (데이터 수집 시 여기에 저장됨)
        """
        self.base_url = base_url
        self.all_data = []  # 수집된 데이터를 저장할 리스트
    
    def fetch_data(self, start_idx: int, end_idx: int) -> tuple:
        """
        API에서 지정된 범위의 데이터를 가져옵니다.
        
        이 메서드는 실제 API 호출을 수행하는 핵심 메서드입니다.
        
        작동 원리:
        1. URL 생성: base_url + "/시작번호/종료번호"
        2. HTTP GET 요청 전송
        3. 받은 XML 데이터 파싱
        4. 전체 데이터 개수 확인 (list_total_count 태그)
        5. 각 <row> 태그의 데이터를 딕셔너리로 변환
        
        Args:
            start_idx (int): 시작 인덱스 (1부터 시작)
                예: 1 → 첫 번째 데이터부터
            end_idx (int): 종료 인덱스 (포함)
                예: 10 → 10번째 데이터까지
                
        Returns:
            tuple: (데이터 리스트, 전체 데이터 개수)
                - 데이터 리스트: [{컬럼명: 값, ...}, {...}] 형태
                - 전체 데이터 개수: API에 저장된 총 데이터 수
                
        예제:
            >>> data, total = collector.fetch_data(1, 10)
            요청 중: 1 ~ 10
              → 수집 완료: 10개 (전체: 1234개)
            >>> print(len(data))  # 10
            >>> print(total)  # 1234
        
        오류 처리:
            - 네트워크 오류: 빈 리스트와 0 반환
            - XML 파싱 오류: 빈 리스트와 0 반환
            - 타임아웃: 10초 후 오류 처리
        """
        # URL 생성: http://주소/1/10 형태
        url = f"{self.base_url}/{start_idx}/{end_idx}"
        
        try:
            print(f"요청 중: {start_idx} ~ {end_idx}")
            
            # API 호출 (최대 10초 대기)
            response = requests.get(url, timeout=10)
            response.raise_for_status()  # HTTP 오류 시 예외 발생
            
            # XML 파싱
            # response.content: 바이트 형태의 XML 데이터
            # ET.fromstring(): XML 문자열을 파싱하여 트리 구조로 변환
            root = ET.fromstring(response.content)
            
            # 전체 데이터 개수 확인
            # XML 구조: <result><list_total_count>1234</list_total_count>...</result>
            total_count_elem = root.find('totalCnt')
            total_count = int(total_count_elem.text) if total_count_elem is not None else 0
            
            # 데이터 추출
            # XML 구조: <result><row><FARM_NM>...</FARM_NM><AREA_LNM>...</AREA_LNM></row>...</result>
            data_list = []
            for row in root.findall('row'):  # 모든 <row> 태그 찾기
                item = {}
                for child in row:  # <row> 안의 모든 자식 태그
                    # child.tag: 태그 이름 (예: FARM_NM)
                    # child.text: 태그 내용 (예: 미사뒷벌텃밭)
                    item[child.tag] = child.text
                data_list.append(item)
            
            print(f"  → 수집 완료: {len(data_list)}개 (전체: {total_count}개)")
            return data_list, total_count
            
        except requests.exceptions.RequestException as e:
            # 네트워크 오류 (연결 실패, 타임아웃 등)
            print(f"  → 오류 발생: {e}")
            return [], 0
        except ET.ParseError as e:
            # XML 형식이 잘못된 경우
            print(f"  → XML 파싱 오류: {e}")
            return [], 0
    
    def collect_all_data(self, batch_size: int = 10, delay: float = 0.5):
        """
        전체 데이터를 수집합니다.
        
        Args:
            batch_size: 한 번에 가져올 데이터 개수 (기본값: 10)
            delay: 요청 간 대기 시간(초) (기본값: 0.5)
        """
        print("=" * 60)
        print("텃밭 정보 API 데이터 수집 시작")
        print("=" * 60)
        
        # 첫 번째 요청으로 전체 데이터 개수 확인
        first_data, total_count = self.fetch_data(1, batch_size)
        
        if total_count == 0:
            print("\n수집할 데이터가 없습니다.")
            return
        
        self.all_data.extend(first_data)
        
        # 나머지 데이터 수집
        current_idx = batch_size + 1
        
        while current_idx <= total_count:
            time.sleep(delay)  # API 부하 방지
            
            end_idx = min(current_idx + batch_size - 1, total_count)
            data_list, _ = self.fetch_data(current_idx, end_idx)
            
            if not data_list:
                print(f"  → 데이터 수집 실패. 다시 시도...")
                time.sleep(2)
                continue
            
            self.all_data.extend(data_list)
            current_idx = end_idx + 1
        
        print("\n" + "=" * 60)
        print(f"수집 완료! 총 {len(self.all_data)}개 데이터 수집")
        print("=" * 60)
    
    def save_to_csv(self, filename: str = "farm_data.csv"):
        """
        수집된 데이터를 CSV 파일로 저장합니다.
        
        Args:
            filename: 저장할 파일명
        """
        if not self.all_data:
            print("저장할 데이터가 없습니다.")
            return
        
        df = pd.DataFrame(self.all_data)
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"\n데이터가 '{filename}' 파일로 저장되었습니다.")
        print(f"저장된 데이터 수: {len(df)}개")
        print(f"컬럼 수: {len(df.columns)}개")
    
    def save_to_excel(self, filename: str = "farm_data.xlsx"):
        """
        수집된 데이터를 Excel 파일로 저장합니다.
        
        Args:
            filename: 저장할 파일명
        """
        if not self.all_data:
            print("저장할 데이터가 없습니다.")
            return
        
        df = pd.DataFrame(self.all_data)
        df.to_excel(filename, index=False, engine='openpyxl')
        print(f"\n데이터가 '{filename}' 파일로 저장되었습니다.")
        print(f"저장된 데이터 수: {len(df)}개")
    
    def get_dataframe(self) -> pd.DataFrame:
        """
        수집된 데이터를 DataFrame으로 반환합니다.
        
        Returns:
            pandas DataFrame
        """
        return pd.DataFrame(self.all_data)
    
    def print_summary(self):
        """데이터 요약 정보를 출력합니다."""
        if not self.all_data:
            print("수집된 데이터가 없습니다.")
            return
        
        df = pd.DataFrame(self.all_data)
        print("\n" + "=" * 60)
        print("데이터 요약")
        print("=" * 60)
        print(f"총 데이터 수: {len(df)}개")
        print(f"컬럼 수: {len(df.columns)}개")
        print(f"\n컬럼 목록:")
        for col in df.columns:
            print(f"  - {col}")
        
        # 지역별 통계
        if 'AREA_LNM' in df.columns:
            print(f"\n지역별 텃밭 수:")
            region_counts = df['AREA_LNM'].value_counts()
            for region, count in region_counts.head(10).items():
                print(f"  - {region}: {count}개")


# 사용 예제
if __name__ == "__main__":
    # API URL 설정
    key = os.getenv('FARMER_KEY')
    BASE_URL = f"http://211.237.50.150:7080/openapi/{key}/xml/Grid_20171122000000000552_1"
    
    # 수집기 생성
    collector = FarmAPICollector(BASE_URL)
    
    # 전체 데이터 수집 (10개씩)
    collector.collect_all_data(batch_size=10, delay=0.5)
    
    # 데이터 요약 출력
    collector.print_summary()
    
    # CSV 파일로 저장
    collector.save_to_csv("텃밭정보_전체데이터.csv")
    
    # Excel 파일로 저장 (선택사항)
    # collector.save_to_excel("텃밭정보_전체데이터.xlsx")
    
    # DataFrame으로 가져오기 (추가 분석용)
    # df = collector.get_dataframe()
    # print(df.head())